In [1]:
import tensorflow as tf
import math
from tensorflow.keras.layers import BatchNormalization


In [1]:
from tensor2tensor.models import transformer
import tensorflow as tf

hparams = transformer.transformer_base()
encoder = transformer.TransformerEncoder(hparams, mode=tf.estimator.ModeKeys.TRAIN)


ModuleNotFoundError: No module named 'tensorflow_addons'

In [6]:
x = tf.range(-5 // 2 + 1, 5 // 2 + 1, dtype=tf.float32)
g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(1, tf.float32), 2))))
g_norm2d = tf.pow(tf.reduce_sum(g), 2)
g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d

In [14]:
tf.nn.top_k(tf.constant([0.3, 0.6, 0.9]), 2, sorted=False)

TopKV2(values=<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.6, 0.9], dtype=float32)>, indices=<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2])>)

In [17]:
tf.roll(tf.constant([[1,2,3,4,5], [4,5,6,7,8], [9,8,7,6,5]]), -1, axis=1)

<tf.Tensor: shape=(3, 5), dtype=int32, numpy=
array([[2, 3, 4, 5, 1],
       [5, 6, 7, 8, 4],
       [8, 7, 6, 5, 9]])>

In [4]:
ema.apply(tf.Variable([1,1,5,3,5,2,2,7,6,6,11,3,3,3,5,5,5,5,5,5,0,0,3,5,6,6,7,9,0,0,0,0,6,6]))

TypeError: 'Variable' object is not iterable.

In [93]:
tf.round(tf.constant([0.3, 0.6, 0.9]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 1.], dtype=float32)>

In [151]:
tf.reshape(tf.constant([1,2,3,4,5,6,7,8,9,0]), [5,2])

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4],
       [5, 6],
       [7, 8],
       [9, 0]])>

In [135]:
zarg,energy

(<tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([ 5,  3,  5,  2,  2,  7,  6,  6, 11,  3,  3,  3,  5,  5,  5,  5,  5,
         5,  0,  0,  3,  5,  6,  6,  7,  9,  0,  0,  0,  0,  6,  6])>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 0, 0])>)

<tf.Tensor: shape=(32,), dtype=int32, numpy=
array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0])>

In [129]:

zarg =   tf.constant([1,1,5,3,5,2,2,7,6,6,11,3,3,3,5,5,5,5,5,5,0,0,3,5,6,6,7,9,0,0,0,0,6,6])
energy = tf.constant([0,0,1,1,1,0,0,1,0,0, 1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,1,1,0,0,1,1,1,0,0])
# z = tf.reshape(y, [-1, 6, 60])
# z = tf.reduce_mean(z, axis=1) #(None, 60)
# zarg = tf.argmax(z, axis=1)


energy_r = tf.roll(energy, 1,axis=-1)
energy_r = tf.concat([[0], energy_r[1:]], axis=-1)


# den = Dense(2, activation='sigmoid')
# energy_r = tf.argmax(den(energy_r))
# energy = tf.argmax(den(energy))
delta_en = energy - energy_r
delta_en_abs = tf.abs(delta_en)

delta_en_csum = tf.math.cumsum(delta_en_abs)

delta_en_csum_pos = tf.squeeze(tf.where(tf.greater(delta_en_csum, 0)), axis=1)

zarg = tf.gather(zarg, delta_en_csum_pos)
delta_en_csum = tf.gather(delta_en_csum, delta_en_csum_pos)
energy = tf.gather(energy, delta_en_csum_pos)
delta_en = tf.gather(delta_en, delta_en_csum_pos)

seg_sum_1 = tf.math.segment_sum(1-energy, delta_en_csum)
seg_sum_where = tf.squeeze(tf.where(tf.greater(seg_sum_1, 0)), axis=1)
seg_sum_1 = tf.gather(seg_sum_1, seg_sum_where)

delta_en_mo = tf.squeeze(tf.where(tf.equal(delta_en, -1)), axis=1)-1
zarg_deta = tf.gather(zarg, delta_en_mo)
seg_sum_usm = tf.math.unsorted_segment_mean(seg_sum_1, zarg_deta, tf.reduce_max(zarg_deta)+1)

# seg_sum_usm_1 = tf.math.unsorted_segment_mean(seg_sum_usm, zarg, 12)

logspace_idx = tf.cast(tf.floor(tf.math.log1p(seg_sum_usm)/math.log(2)), tf.int32) + 3
use_identity = tf.cast(seg_sum_usm <= 4, tf.int32)
seg_sum_usm = tf.cast(seg_sum_usm, tf.int32)
combined_idx = use_identity * seg_sum_usm + (1 - use_identity) * logspace_idx


clipped = tf.clip_by_value(combined_idx, 0, 9)

# emb = tf.get_variable('silence_embeddings', shape=[10, 128])
# return tf.gather(emb, clipped)

InvalidArgumentError: data.shape = [12] does not start with segment_ids.shape = [32] [Op:UnsortedSegmentSum]

In [138]:
clipped

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 3])>

In [139]:
zarg_deta

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([ 5,  7, 11,  5,  0,  3,  7,  0])>

In [137]:
zarg_deta

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([ 5,  7, 11,  5,  0,  3,  7,  0])>

In [136]:
clipped

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 3])>

In [107]:
energy

<tf.Tensor: shape=(12,), dtype=int32, numpy=array([1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0])>

In [113]:
delta_en_csum

<tf.Tensor: shape=(18,), dtype=int32, numpy=array([1, 1, 1, 2, 2, 3, 4, 4, 5, 6, 6, 6, 7, 7, 8, 8, 8, 8])>

In [86]:
tf.math.log1p(seg_sum_usm)/math.log(2)

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.      , 0.      , 1.      , 2.321928, 2.807355, 3.169925,
       6.022368, 7.      , 0.      , 6.629357], dtype=float32)>

In [87]:
logspace_idx = tf.cast(tf.floor(tf.math.log1p(seg_sum_usm)/math.log(2)), tf.int32) + 3

In [88]:
use_identity = tf.cast(seg_sum_usm <= 4, tf.int32)

In [89]:
seg_sum_usm = tf.cast(seg_sum_usm, tf.int32)

In [90]:
use_identity * seg_sum_usm + (1 - use_identity) * logspace_idx

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 0,  0,  1,  4,  5,  6,  9, 10,  0,  9])>